In [1]:
import ee

In [2]:
ee.Initialize()


In [7]:
aoi=ee.Geometry.Polygon( [
[46.41388398001249,24.474133938125608],
[46.99272614309843,24.474133938125608],
[46.99272614309843,25.04372308127158],
[46.41388398001249,25.04372308127158],
[46.41388398001249,24.474133938125608]
])  #new RUH-for ndvi values

In [4]:
aoi=ee.Geometry.Polygon( [
[39.25879949255623,21.88984377820288],
[39.26532262488045,21.88984377820288],
[39.26532262488045,21.894861167298092],
[39.25879949255623,21.894861167298092],
[39.25879949255623,21.88984377820288]
])  #uj

In [5]:
# Function to mask clouds in Landsat 8 images
def maskL8Clouds(image):
    qa = image.select('pixel_qa')
    # Cloud bit mask and shadow bit mask - these values depend on the Landsat 8 QA band
    cloudBitMask = 1 << 5
    shadowBitMask = 1 << 3

    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(shadowBitMask).eq(0))
    return image.updateMask(mask).divide(10000)  # Scale the pixel values

# Filter the Landsat 8 ImageCollection for the specified date range, AOI, and apply cloud masking
landsatCollection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
    .filterDate('2023-06-01', '2024-01-01') \
    .filterBounds(aoi) \
    .map(maskL8Clouds) \
    .select(['B4', 'B5']) \
    .median()  # Compute the median of the filtered collection

# Clip the median image to the AOI
clippedImage = landsatCollection.clip(aoi)

# Function to cast selected bands to Float32
def castImage(image):
    return image.toFloat()

# Cast the selected bands of the clipped image to Float32
castedImage = castImage(clippedImage)

# Example export parameters (make sure to replace 'aoi.getInfo()['coordinates']' with your AOI coordinates if necessary)
task = ee.batch.Export.image.toDrive(**{
    'image': castedImage,
    'description': 'uj_Landsat-8_2',
    'scale': 30,  # Landsat 8 has a different resolution; 30 meters for bands B4 and B5
    'region': aoi.getInfo()['coordinates'],  # Make sure this is correctly defined for your AOI
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})
task.start()
